In [1]:
%load_ext autoreload
%autoreload 2

In this implementation we use the California housing dataset to predict housing prices.

In [2]:
from typing import List, Union
import math
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
from tensorflow.python.framework import tensor_shape

import tensorflow_probability as tfp

from bdl.pbp.gamma_initializer import ReciprocalGammaInitializer
from bdl.pbp.math import safe_div, non_negative_constraint


RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

2022-04-12 08:31:11.868039: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# TODO: replace with California housing dataset
X, y = datasets.load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [4]:
def ensure_input(x, dtype, input_shape):
    x = tf.constant(x, dtype=dtype)
    call_rank = tf.rank(tf.constant(0, shape=input_shape, dtype=dtype)) + 1
    if tf.rank(x) < call_rank:
        x = tf.reshape(x, [-1, * input_shape.as_list()])
    return x


def ensure_output(y, dtype, output_dim):
    output_rank = 2
    y = tf.constant(y, dtype=dtype)
    if tf.rank(y) < output_rank:
        y = tf.reshape(y, [-1, output_dim])
    return y

In [5]:
def normalize(x, y, output_shape):
    x = ensure_input(x, tf.float32, x.shape[1])
    y = ensure_output(y, tf.float32, output_shape)
    mean_X_train, mean_y_train, std_X_train, std_y_train = get_mean_std_x_y(x, y)
    x = (x - np.full(x.shape, mean_X_train)) / np.full(x.shape, std_X_train)
    y = (y - mean_y_train) / std_y_train
    return x, y


def get_mean_std_x_y(x, y):
    std_X_train = np.std(x, 0)
    std_X_train[std_X_train == 0] = 1
    mean_X_train = np.mean(x, 0)
    std_y_train = np.std(y)
    if std_y_train == 0.0:
        std_y_train = 1.0
    mean_y_train = np.mean(y)
    return mean_X_train, mean_y_train, std_X_train, std_y_train

In [6]:
x, y = normalize(X_train, y_train, 1)

In [7]:
class PBPLayer(tf.keras.layers.Layer):
    def __init__(self, units: int, dtype=tf.float32, *args, **kwargs):
        super().__init__(dtype=tf.as_dtype(dtype), *args, **kwargs)
        self.units = units

    def build(self, input_shape):
        input_shape = tensor_shape.TensorShape(input_shape)
        last_dim = tensor_shape.dimension_value(input_shape[-1])
        self.input_spec = tf.keras.layers.InputSpec(min_ndim=2, axes={-1: last_dim})
        self.inv_sqrtV1 = tf.cast(1.0 / tf.math.sqrt(1.0 * last_dim + 1), dtype=self.dtype)
        self.inv_V1 = tf.math.square(self.inv_sqrtV1)

        over_gamma = ReciprocalGammaInitializer(6.0, 6.0)
        self.kernel_m = self.add_weight(
            "kernel_mean",
            shape=[last_dim, self.units],
            initializer=tf.keras.initializers.HeNormal(),
            dtype=self.dtype,
            trainable=True,
        )
        self.kernel_v = self.add_weight(
            "kernel_variance",
            shape=[last_dim, self.units],
            initializer=over_gamma,
            dtype=self.dtype,
            trainable=True,
        )
        self.bias_m = self.add_weight(
            "bias_mean",
            shape=[self.units],
            initializer=tf.keras.initializers.HeNormal(),
            dtype=self.dtype,
            trainable=True,
        )
        self.bias_v = self.add_weight(
            "bias_variance",
            shape=[self.units],
            initializer=over_gamma,
            dtype=self.dtype,
            trainable=True,
        )
        self.Normal = tfp.distributions.Normal(
            loc=tf.constant(0.0, dtype=self.dtype),
            scale=tf.constant(1.0, dtype=self.dtype),
        )
        self.built = True

    @tf.function
    def apply_gradient(self, gradient):
        dlogZ_dkm, dlogZ_dkv, dlogZ_dbm, dlogZ_dbv = gradient

        # Kernel
        self.kernel_m.assign_add(self.kernel_v * dlogZ_dkm)
        new_kv = self.kernel_v - (tf.math.square(self.kernel_v) * (tf.math.square(dlogZ_dkm) - 2 * dlogZ_dkv))
        self.kernel_v.assign(non_negative_constraint(new_kv))

        # Bias
        self.bias_m.assign_add(self.bias_v * dlogZ_dbm)
        new_bv = self.bias_v - (tf.math.square(self.bias_v) * (tf.math.square(dlogZ_dbm) - 2 * dlogZ_dbv))
        self.bias_v.assign(non_negative_constraint(new_bv))

    @tf.function
    def _sample_weights(self):
        eps_k = self.Normal.sample(self.kernel_m.shape)
        std_k = tf.math.sqrt(tf.maximum(self.kernel_v, tf.zeros_like(self.kernel_v)))
        W = self.kernel_m + std_k * eps_k

        eps_b = self.Normal.sample(self.bias_m.shape)
        std_b = tf.math.sqrt(tf.maximum(self.bias_v, tf.zeros_like(self.bias_v)))
        b = self.bias_m + std_b * eps_b
        return W, b

    @tf.function
    def call(self, x: tf.Tensor):
        W, b = self._sample_weights()
        return (tf.tensordot(x, W, axes=[1, 0]) + tf.expand_dims(b, axis=0)) * self.inv_sqrtV1

    @tf.function
    def predict(self, previous_mean: tf.Tensor, previous_variance: tf.Tensor):
        mean = (
            tf.tensordot(previous_mean, self.kernel_m, axes=[1, 0])
            + tf.expand_dims(self.bias_m, axis=0)
        ) * self.inv_sqrtV1

        variance = (
            tf.tensordot(previous_variance, tf.math.square(self.kernel_m), axes=[1, 0])
            + tf.tensordot(tf.math.square(previous_mean), self.kernel_v, axes=[1, 0])
            + tf.expand_dims(self.bias_v, axis=0)
            + tf.tensordot(previous_variance, self.kernel_v, axes=[1, 0])
        ) * self.inv_V1

        return mean, variance

In [8]:
class PBPReLULayer(PBPLayer):
    @tf.function
    def call(self, x: tf.Tensor):
        """Calculate deterministic output"""
        # x is of shape [batch, prev_units]
        x = super().call(x)
        z = tf.maximum(x, tf.zeros_like(x))  # [batch, units]
        return z

    @tf.function
    def predict(self, previous_mean: tf.Tensor, previous_variance: tf.Tensor):
        ma, va = super().predict(previous_mean, previous_variance)
        mb, vb = get_mb_vb(ma, va, self.Normal)
        return mb, vb

In [9]:
def get_mb_vb(ma, va, normal):
    _sqrt_v = tf.math.sqrt(tf.maximum(va, tf.zeros_like(va)))
    _alpha = safe_div(ma, _sqrt_v)
    _inv_alpha = safe_div(tf.constant(1.0, dtype=_alpha.dtype), _alpha)
    _cdf_alpha = normal.cdf(_alpha)
    _gamma = tf.where(
        _alpha < -30,
        -_alpha + _inv_alpha * (-1 + 2 * tf.math.square(_inv_alpha)),
        safe_div(normal.prob(-_alpha), _cdf_alpha),
    )
    _vp = ma + _sqrt_v * _gamma
    mb = _cdf_alpha * _vp
    vb = mb * _vp * normal.cdf(-_alpha) + _cdf_alpha * va * (
            1 - _gamma * (_gamma + _alpha)
    )
    return mb, vb

Our model will be a model with two ReLU layers with two 50 hidden units and a linear last layer.

In [10]:
units = [50, 50, 1]

def build_layers(last_shape, units):
    layers = []
    for unit in units[:-1]:
        layer = PBPReLULayer(unit)
        layer.build(last_shape)
        layers.append(layer)
        last_shape = unit
    layer = PBPLayer(units[-1])
    layer.build(last_shape)
    layers.append(layer)
    return layers

In [11]:
units = [50, 50, 1]
layers = build_layers(X_train.shape[1], [50, 50, 1])

We can now instantiate our PBP model. We see that we define a few more variables that we need for PBP. ...

In [12]:
pi = tf.math.atan(tf.constant(1.0, dtype=tf.float32)) * 4
LOG_INV_SQRT2PI = -0.5 * tf.math.log(2.0 * pi)


class PBP:
    def __init__(
        self,
        layers: List[tf.keras.layers.Layer],
        dtype: Union[tf.dtypes.DType, np.dtype, str] = tf.float32
    ):
        self.alpha = tf.Variable(6.0, trainable=True, dtype=dtype)
        self.beta = tf.Variable(6.0, trainable=True, dtype=dtype)
        self.layers = layers
        self.Normal = tfp.distributions.Normal(
            loc=tf.constant(0.0, dtype=dtype),
            scale=tf.constant(1.0, dtype=dtype),
        )
        self.Gamma = tfp.distributions.Gamma(concentration=self.alpha, rate=self.beta)

    def fit(self, x, y, batch_size: int = 16, n_epochs: int = 1):
        data = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size)
        for epoch_index in range(n_epochs):
            print(f"{epoch_index=}")
            for x_batch, y_batch in data:
                diff_square, v, v0 = self.update_gradients(x_batch, y_batch)
                alpha, beta = update_alpha_beta(self.alpha, self.beta, diff_square, v, v0)
                self.alpha.assign(alpha)
                self.beta.assign(beta)

    @tf.function
    def predict(self, x: tf.Tensor):
        m, v = x, tf.zeros_like(x)
        for layer in self.layers:
            m, v = layer.predict(m, v)
        return m, v

    @tf.function
    def update_gradients(self, x, y):
        trainables = [layer.trainable_weights for layer in self.layers]
        with tf.GradientTape() as tape:
            tape.watch(trainables)
            m, v = self.predict(x)
            v0 = v + safe_div(self.beta, self.alpha - 1)
            diff_square = tf.math.square(y - m)
            logZ0 = logZ(diff_square, v0)
        grad = tape.gradient(logZ0, trainables)
        for l, g in zip(self.layers, grad):
            l.apply_gradient(g)
        return diff_square, v, v0

In [13]:
model = PBP(layers)

In [14]:
def update_alpha_beta(alpha, beta, diff_square, v, v0):
    alpha1 = alpha + 1
    v1 = v + safe_div(beta, alpha)
    v2 = v + beta / alpha1
    logZ2_logZ1 = logZ1_minus_logZ2(diff_square, v1=v2, v2=v1)
    logZ1_logZ0 = logZ1_minus_logZ2(diff_square, v1=v1, v2=v0)
    logZ_diff = logZ2_logZ1 - logZ1_logZ0
    Z0Z2_Z1Z1 = safe_exp(logZ_diff)
    # Must update beta first
    # Extract larger exponential
    pos_where = safe_exp(logZ2_logZ1) * (alpha1 - safe_exp(-logZ_diff) * alpha)
    neg_where = safe_exp(logZ1_logZ0) * (Z0Z2_Z1Z1 * alpha1 - alpha)
    beta_denomi = tf.where(logZ_diff >= 0, pos_where, neg_where)
    beta = safe_div(beta, tf.maximum(beta_denomi, tf.zeros_like(beta)))

    alpha_denomi = Z0Z2_Z1Z1 * safe_div(alpha1, alpha) - 1.0

    alpha = safe_div(
        tf.constant(1.0, dtype=alpha_denomi.dtype),
        tf.maximum(alpha_denomi, tf.zeros_like(alpha)),
    )

    return alpha, beta


@tf.function
def logZ(diff_square: tf.Tensor, v: tf.Tensor):
    v0 = v + 1e-6
    return tf.reduce_sum(
        -0.5 * (diff_square / v0) + LOG_INV_SQRT2PI - 0.5 * tf.math.log(v0)
    )


@tf.function
def logZ1_minus_logZ2(diff_square: tf.Tensor, v1: tf.Tensor, v2: tf.Tensor):
    return tf.reduce_sum(
        -0.5 * diff_square * safe_div(v2 - v1, v1 * v2)
        - 0.5 * tf.math.log(safe_div(v1, v2) + 1e-6)
    )

In [15]:
@tf.function
def safe_div(x: tf.Tensor, y: tf.Tensor, eps: tf.Tensor = tf.constant(1e-6)):
    _eps = tf.cast(eps, dtype=y.dtype)
    return x / (tf.where(y >= 0, y + _eps, y - _eps))


@tf.function
def safe_exp(x: tf.Tensor, BIG: tf.Tensor = tf.constant(20)):
    return tf.math.exp(tf.math.minimum(x, tf.cast(BIG, dtype=x.dtype)))


@tf.function
def non_negative_constraint(x: tf.Tensor):
    return tf.maximum(x, tf.zeros_like(x))

In [16]:
model.fit(x, y, batch_size=1, n_epochs=1)

epoch_index=0


2022-04-12 08:31:14.032847: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


## Predict

In [17]:
# normalise test set
mean_X_train, mean_y_train, std_X_train, std_y_train = get_mean_std_x_y(X_train, y_train)
# We normalize the test set
X_test = (X_test - np.full(X_test.shape, mean_X_train)) / np.full(X_test.shape, std_X_train)

# perform inference on normalised data
X_test = ensure_input(X_test, tf.float32, X_test.shape[1])
m, v = model.predict(X_test)
v_noise = (model.beta / (model.alpha - 1) * std_y_train**2)
# add mean and std back to m and v
m = m * std_y_train + mean_y_train
v = v * std_y_train**2

# transform back to original space
m = np.squeeze(m.numpy())
v = np.squeeze(v.numpy())
v_noise = np.squeeze(v_noise.numpy().reshape(-1, 1))

# calculate rmse
rmse = np.sqrt(np.mean((y_test - m) ** 2))
# calculate log-likelihood
test_ll = np.mean(
    -0.5 * np.log(2 * math.pi * v)
    - 0.5 * (y_test - m) ** 2 / v
)
# calculate log-likelihood with v_noise
test_ll_with_vnoise = np.mean(
    -0.5 * np.log(2 * math.pi * (v + v_noise))
    - 0.5 * (y_test - m) ** 2 / (v + v_noise)
)
rmse, test_ll, test_ll_with_vnoise

(6.651302891855423, -3.2725913393996726, -3.2353817613326985)